# Machine Learning Engineer Nanodegree
## Capstone Proposal

Brian Palmer  
September 14, 2017

## Proposal

### Domain Background

Some of my primary areas of interest in machine learning are:

1. Using reinforcement learning to train AIs to solve complex tasks as they perform the tasks (as opposed to using supervised or unsupervised learning on pre-collected data sets).
2. Neural networks and AIs that can interpret and operate directly on high-dimensional sensor data, such as images or video streams.
3. Robotics and software for hardware interacting in real-world environments, such as in warehouses, factories, and self-driving cars/drones.

In recent years, much progress has been made in these areas. For instance, the [Deep Q Networks paper](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) outlines a deep convolutional neural network that directly reads video input, Atari video games, and learns discrete control policies to play many games at a human level. The subsequent [Deep Deterministic Policy Gradient paper](https://arxiv.org/pdf/1509.02971v2.pdf) solves continuous control problems (which are common in robotics) by building on the DQN work and adding an actor-critic architecture which can operate on high-dimensional continuous outputs. The [K-FAC paper](https://arxiv.org/pdf/1503.05671.pdf) outlines a method of using the natural gradient, based on the Fisher information matrix, to optimize a neural network in fewer timesteps.

Last year, the [Asynchronous Methods for Deep Reinforcement Learning paper](https://arxiv.org/pdf/1602.01783.pdf) outlined an asynchronous, parallel reinforcement learning agent that can outperform these earlier agents in a shorter period of time even on just a multi-core CPU, and supports continuous outputs as well as high-dimensional inputs. This architecture is called Asynchronous Advantage Actor-Critic (A3C). Later it was recognized that the asynchronous nature of the architecture was not important, and a deterministic version called Advantage Actor-Critic (A2C) has become popular.

### Problem Statement

I propose to implement a machine learning agent that can use reinforcement learning with deep neural networks to solve a set of both discrete and continuous action space problems in the [OpenAI Gym](https://gym.openai.com/) environments. The set of environments to be solved are:

* [Lunar Lander Continuous](https://gym.openai.com/envs/LunarLanderContinuous-v2)
* [Puck World](https://gym.openai.com/envs/PuckWorld-v0)
* [Bipedal Walker Hardcore](https://gym.openai.com/envs/BipedalWalkerHardcore-v2)
* [Car Racing v0](https://gym.openai.com/envs/CarRacing-v0)
* [Atari Space Invaders](https://gym.openai.com/envs/SpaceInvaders-v0)

Some of these problems, such as Space Invaders, are well-explored in the literature, while other problems such as Car Racing and Bipedal Walker are less explored. My goal is to implement a working agent that can learn and solve all these environments using the same architecture, ideally without tuning hyperparameters to each environment.

"Solved" means different things for different Gym environments, but typically it involves reaching a goal state in a certain number of timesteps, or avoiding a failure condition for a number of timesteps. The Gym framework will determine whether the agent has solved the environment as part of the training/testing process. A few environments, such as Space Invaders, are open-ended without a "solved" state. For those environments I will define a threshold reward score above which I will consider the environment solved.

* For _Lunar Lander_: scoring 200 points.
* For _Puck World_: TBD, scoring a set number of points.
* For _Bipedal Walker Hardcore_: scoring 300 points in 2000 time steps.
* For _Car Racing_: scoring 900 points on each track.
* For _Space Invaders_: TBD, scoring a set number of points.

**note:** originally my plan for these open-ended environments was to benchmark my agent against other agents on the Gym leaderboards, but as I was finishing up this proposal doc, OpenAI took down the leaderboards, [apparently for good](https://twitter.com/gdb/status/908103679810584577).

### Datasets and Inputs

Because this is a Reinforcement Learning problem, rather than traditional Supervised or Unsupervised learning, the input states are generated during training as the simulations run, rather than being pre-gathered.

Some of these environments have low-dimensional state spaces consisting of a few (less than 100) input variables. These often correspond to position, velocity, angle, and other physical characteristics. Other environments use the raw pixel data as input, which is of much higher dimension (on the order of thousands of input variables). I will be operating against both types of environments.

Some of these gym environments take one discrete (integer) action as output, often corresponding to a key press such as a joystick button for the Atari environment. Others require multiple continuous (floating point) action outputs, such as the Car Racing and Bipedal Walker environments.

### Solution Statement

I plan to solve these environments by implementing my own Advantage Actor-Critic (A2C) machine learning agent. To be clear, this is not novel work or research-oriented, there are public A2C implementations already available. My ultimate goal is to better understand the state of the art in this area, so I will not utilize any of these existing agents or source code. Instead I will work from the research papers linked above, along with any useful blog posts or other relevant descriptions I find. I plan to produce a high-quality implementation that could be understood and extended by others.

There will also be significant work in tuning the agent and its hyperparameters even once it has been written, in order to get passable results.

In addition to scoring each environment, I will also be tracking other metrics about the agent, including but not limited to:

* Number of timesteps and sessions required for training.
* Computational cost of training, based on wall-clock time.
* Stability of the solution. I will measure this by training the agent on each environment multiple times.

### Benchmark Model

As part of writing this proposal I implemented a basic Deep Q-Learning Agent based on the first paper linked above, using Python3 and [Keras](https://keras.io). I plan to use this agent as my benchmark, tracking the same metrics that I track for my A2C agent and comparing results. Note that the basic DQL agent does not support continuous action spaces without further extensions, so I will not be able to test it on the "Bipedal Walker" and "Car Racing" environments.

I expect my A2C agent to train in fewer timesteps and less wall-clock time, and hopefully score better in the envrionments as well.

### Evaluation Metrics

There are at least four metrics that I will use to evaluate both my DQL benchmark agent and my A2C agent, in each of the environments:

1. Total score (reward).
2. Number of timestamps before a solution is reached, for each training of the agent. This is a simple counter provided by the Gym environment. The primary reason that this is so important isn't just because more timestamps means a longer training time. In robotics problems, it's very important to be efficient in the amount of training required, since training has real-world costs.
3. Relative stability of the learning. This can be estimated by a simple mean, or by measuring the variance in the first metric across all the trainings for the agent, for a given environment. In other words, given a series of training sessions $x$, with $x_i$ being the number of timestamps before the environment is solved for training session $i$, the learning stability can be estimated as $\frac{\sum{(x_i-\bar{x})^2}}{n-1}$.
4. Two computational metrics: wall clock time, and RAM used.

### Project Design

I plan to implement my agent using Python 3 with [Tensorflow](https://www.tensorflow.org). I wrote my DQL agent using Keras, and I have used Keras in a few projects now, but I'd like to use Tensorflow directly for my A2C agent since I believe that it will give me more insight into the actual algorithms being run and how all the pieces tie together.

A quick, informal summary of my Deep Q-Learning benchmark agent:

Initialize replay memory deque $R$ to capacity $N$. Initialize action-value network $Q$ and target network $\hat{Q}$ with random weights. Then, for each training step:

1. With probability `epsilon` select a random action $a_t$. Otherwise select the highest-rated action predicted by $Q$.
2. Execute the chosen action $a_t$ and observe reward $r_t$ and next state $s_{t+1}$.
3. Store $(s_t, a_t, r_t, s_{t+1})$ into $R$.
4. Sample a random subset of the memories stored in $R$, and for each memory:
    1. Estimate the reward $v$ as $r_t + \gamma * amax(\hat{Q}(s_{t+1}))$.
    2. Run backpropagation (fit) $Q$ using the estimated reward $v$ as the value for $a_t$ at $s_t$.
5. Every $C$ timesteps, reset $\hat{Q} = Q$.

The Advantage Actor-Critic agent architecture is quite different. The neural network itself has two output layers, called the actor and the critic. The actor works in a similar manner to the Q-Learning agent, learning a policy and selecting actions. The critic learns a value function and assigns the value to the states visited.

In A2C we spawn $N$ environments in parallel. There is a global copy of the neural network, and each environment also gets its own copy. Then we run in a loop for each environment:

1. For $t_{max}$ steps, or until a terminal state is reached:
    1. Execute the local copy of the network using the current state $s_t$ as input.
    2. Select action values $a_t$ stochastically based on the probabilities returned by the local copy of the actor network. Receive reward $r_i$ and new state $s_{t+1}$.
    3. If this is the first step in this batch, initialize $R$ to the estimated reward given by the local copy of the critic network. Otherwise, $R = r_i + \gamma R$.
2. Accumulate the gradients for each step for both actor and critic and apply them to the global network.
3. Copy the global network back into the actor's local copy, unless terminated.

Typically $t_{max}$ is quite small, on the order of 5 steps.

#### Notes

Some of the gym environments have low-dimensional inputs, while others use raw pixels for input and thus have much higher dimensionality. While the ultimate goal is to have an architecture that can solve all the environments without any changes, I am making one concession: for pixel input the architecture will include some number of Convolutional Neural Network layers, while for low-dimensional environments these layers will be omitted.

So far I have been running these agents locally, but I will likely spin up at least a couple AWS instances so that I can run the agents at a larger scale, especially since I am going to be training each agent on each environment multiple times, in order to gather the data on training stability. I have extensive AWS experience so I don't forsee any difficulties here.